In [ ]:
import numpy as np
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

This file provides some illustration of the generation of sets of Points.
We can create the set of Points which covers a Box (in any space dimension). 
For sake of demonstration, this is demonstrated in 2-D.

We can create the set by drawing samples at random within the Box extension

In [ ]:
nech = 1000
db = gl.Db.createFromBox(nech,[0,0],[100,100],4324);
gp.point(db,size=5,title="Uniform Poisson")
plt.axis("equal") 
print("The number of samples =",db.getSampleNumber())

We now generate a positive Regionalized variable over a grid that will serve as a density.

In [ ]:
# Creating a Grid
grid = gl.DbGrid.create(nx=[100,100])

# Creating the isotropic Model
m = gl.Model()
cov = gl.CovAniso.createIsotropic(ctxt=m.getContext(),type=gl.ECov.BESSEL_K,range=20,param=2)
m.addCov(cov)

# Simulating a Gaussian variable using an isotropic Model then turn positive (exponentiation)
gl.simtub(None,grid,m,nbtuba=400)
a = grid["Simu"]
a[np.where(a < 0)[0]] = 0
grid["Simu"] = a
gp.plot(grid,title="Density variable")

Now we define the set of Points according to different models. The first one is according to Poisson process (i.e. points randomly located within a Box.

In [ ]:
db = gl.Db.createFromDbGrid(nech,grid,4324)
gp.point(db,size=5,title="Regionalized Poisson")
plt.axis("equal") 
print("Number of samples =",db.getSampleNumber())

We now try the repulsion on the Uniform Poisson

In [ ]:
db = gl.Db.createFromBox(nech,[0,0],[100,100],seed=43243,flag_exact=True,flag_repulsion=True,range=5.,beta=100);
gp.point(db,size=5,title="Uniform Poisson with Repulsion")
plt.axis("equal") 
print("The number of samples =",db.getSampleNumber())

Combining the repulsion with the regionalized Poisson internsity

In [ ]:
db = gl.Db.createFromDbGrid(nech,grid,seed=43243,flag_exact=True,flag_repulsion=True,range=5.,beta=100)
gp.point(db,size=5,title="Regionalized Poisson with Repulsion")
plt.axis("equal") 
print("Number of samples =",db.getSampleNumber())

In [ ]:
import gstlearn.plot as gp

def fa(x,y,a,b):
    return a*x + b*y

def spirale(db,a=0,b=-1.4,c=1.,d=1.,plot = False):
    x1c = np.array(db.getColumn("x1")) #getColumn ou mieux getCoords coords = workingDb.getCoords()
    x2c = np.array(db.getColumn("x2")) 
    u1=fa(x1c-50,x2c-50,a,b)
    u2=fa(x1c-50,x2c-50,c,d)
    shape = db.getNXs()
    norm = np.sqrt(u1**2+u2**2)
    ind = norm>0
    theta = np.zeros_like(norm)
    theta[norm>0] = np.arccos(u2[ind]/norm[ind])/np.pi*180*np.sign(u1[ind])
    x1c=x1c.reshape(shape)
    x2c=x2c.reshape(shape)
    u1=u1.reshape(shape)
    u2=u2.reshape(shape)
    if plot:
        plt.quiver(x1c,x2c,u1,u2)
        plt.axis("equal")
        plt.show()
    return theta


# Draw a vector field

# In[ ]:


#Création de la grille de représentation
workingDbc = gl.DbGrid.create([10,10],[10,10])
spirale(workingDbc,plot=False);


resultDb = gl.DbGrid.create([101,101],[1,1]) 
x1 = np.array(resultDb.getColumn("x1")) #getColumn ou mieux getCoords coords = workingDb.getCoords()
x2 = np.array(resultDb.getColumn("x2")) 
theta = spirale(resultDb)
Scale1=100*np.ones(theta.size)
Scale2=5*np.ones(theta.size)
iatt = resultDb.addColumns(Scale1,"Scale1")
iatt = resultDb.addColumns(Scale2,"Scale2")
iatt = resultDb.addColumns(theta,"theta")

resultDb.setLocators(['theta','Scale1','Scale2'],gl.ELoc.NOSTAT)
resultDb["const"] = np.ones_like(resultDb["theta"])
resultDb.setLocator("const",gl.ELoc.Z)
gp.grid(resultDb,"theta")


In [ ]:
db = gl.Db.createFromDbGrid(1000,resultDb,seed=43243,flag_exact=True,flag_repulsion=True,range=1,beta=2000);
gp.point(db,size=5,title="Uniform Poisson with Repulsion")
plt.axis("equal") 
print("The number of samples =",db.getSampleNumber())